## Tutotial: 
 
https://medium.com/data-hackers/web-scraping-com-python-para-pregui%C3%A7osos-unindo-beautifulsoup-e-selenium-parte-1-9677fc5e2385

1. Lendo a Página: 

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [ ]:
html = urlopen('https://receita.economia.gov.br/acesso-rapido/agenda-tributaria/agenda-tributaria-2020/agenda-tributaria-janeiro-2020/dia-06-01-2020')
scrap = BeautifulSoup(html, 'html.parser')
scrap.prettify()

2. Buscando o titulo:

In [26]:
scrap.find('title')

<title>Dia 06/01/2020 —  Receita Federal </title>

In [27]:
scrap.title.text

'Dia 06/01/2020 —  Receita Federal '

3. Extraindo a tabela

* Dentro desta página temos uma tabela, onde:
  * `< tbody >` : corpo da tabela
    * `< tr class = 'even'` : cada linha
  
Para acessar todos os elementos `<tr>` que possuem a classe `‘even’`, ou seja, as linhas da tabela, podemos fazer:

In [37]:
linhas = scrap.find_all('tr', {'class':'even'})  # vemos que assim ele não encontra

Inspecionando, vemos que em cada linha existem três filhas, que são as tags `<td>` que representa as colunas. Queremos coletar o texto dentro delas:
1. usando um for e buscando o text dentro do elemento
2. usando a função findChildren('td') que armazena as tags < td > que estão da < tr class = 'even'

In [ ]:
# 1. 
for i in linhas:
    print(i.text)
    

# 2.
for i in linhas:
    filhas = i.findChildren("td")
    print(filhas[0].text)
    print(filhas[1].text)
    print(filhas[2].text)
    
    
# 8053
# IRRF - Títulos de Renda Fixa - Pessoa Física
# 21 a 31/dezembro/2019

Com isso, vemos que o txt contido corresponde aos headers da tabela do site da Receita:
* Código DARF;
* Descrição do tributo/contribuição; 
* Período do Fato Gerador

In [53]:
codigo = []
descricao = []
periodo = []

# coletando as informações dentro das listas:

for i in linhas:
    filhas = i.findChildren("td")
    codigo.append(filhas[0].text)
    descricao.append(filhas[1].text)
    periodo.append(filhas[2].text)

print(codigo[:2], '\n')
print(descricao[:2], '\n')
periodo[:2]

['8053', '3426'] 

['IRRF - Títulos de Renda Fixa - Pessoa Física', 'IRRF - Títulos de Renda Fixa - Pessoa Jurídica'] 



['21 a 31/dezembro/2019', '21 a 31/dezembro/2019']

4. Inserindo as linhas criadas dentro de um DF (pandas)

In [57]:
import pandas as pd

df = pd.DataFrame({'Código': codigo, 'Descrição': descricao, 'Período': periodo})
df.head()

,Código,Descrição,Período
0,8053,IRRF - Títulos de Renda Fixa - Pessoa Física,21 a 31/dezembro/2019
1,3426,IRRF - Títulos de Renda Fixa - Pessoa Jurídica,21 a 31/dezembro/2019
2,6800,IRRF - Fundo de Investimento - Renda Fixa,21 a 31/dezembro/2019
3,6813,IRRF - Fundo de Investimento em Ações,21 a 31/dezembro/2019
4,5273,IRRF - Operações de SWAP,21 a 31/dezembro/2019


### Para salvar a tabela em um arquivo excel:

In [58]:
df.to_excel('dados_receita.xlsx')